# Demo of ATS1 benchmark using Jupyter Notebook

This is a demo of FalCAuN on Jupyter Notebook using the [kotlin-jupyter kernel](https://github.com/Kotlin/kotlin-jupyter). This demo assumes that `jupyter` is executed with the following environmental variables.

- `JAVA_HOME` (the java home for Java 21)
- `KOTLIN_JUPYTER_JAVA_OPTS="-Djava.library.path=$MATLAB_HOME/bin/maca64/:$MATLAB_HOME/bin/maci64:$MATLAB_HOME/bin/glnxa64"`

## Dependent packages

This notebook depends on FalCAuN-core and FalCAuN-matlab

In [1]:
@file:DependsOn("net.maswag.falcaun:FalCAuN-core:1.0-SNAPSHOT")
@file:DependsOn("net.maswag.falcaun:FalCAuN-matlab:1.0-SNAPSHOT")

## Automatic transmission model

The following shows the common configuration to run the Automatic Transmission benchmark [Hoxha et al., ARCH@CPSWeek 2014].

* [Hoxha et al., ARCH@CPSWeek 2014]: *Benchmarks for Temporal Logic Requirements for Automotive Systems*, ARCH@CPSWeek 2014, Bardh Hoxha, Houssam Abbas, Georgios E. Fainekos

In [2]:
import net.maswag.falcaun.*

val initScript = """
versionString = version('-release');
oldpath = path;
path(strcat(userpath, '/Examples/R', versionString, '/simulink_automotive/ModelingAnAutomaticTransmissionControllerExample/'), oldpath);

mdl = 'Autotrans_shift';
load_system(mdl);
"""
val paramNames = listOf("throttle", "brake")
val signalStep = 1.0
val simulinkSimulationStep = 0.0025

In [3]:
// Load the automatic transmission model. This must be manually closed!!
val sul = SimulinkSUL(initScript, paramNames, signalStep, simulinkSimulationStep)

## Definition of the STL properties

In [8]:
// Define the input and output mappers
val throttleValues = listOf(0.0, 100.0)
val brakeValues = listOf(0.0, 325.0)
val inputMapper = InputMapperReader.make(listOf(throttleValues, brakeValues))
val velocityValues = listOf(20.0, 22.5, 25.0, 27.5, 30.0, null)
val accelerationValues = listOf(null)
val gearValues = listOf(2.0, 3.0, null)
val outputMapperReader = OutputMapperReader(listOf(velocityValues, accelerationValues, gearValues))
outputMapperReader.parse()
val signalMapper = SimpleSignalMapper()
val mapper =
    NumericSULMapper(inputMapper, outputMapperReader.largest, outputMapperReader.outputMapper, signalMapper)

In [9]:
import net.maswag.falcaun.TemporalLogic.STLCost;

// Define the STL properties
val stlFactory = STLFactory()
val stlList: List<STLCost> = listOf(
    "[]((signal(2) == 3) -> signal(0) > 20)",
    "[]((signal(2) == 3) -> signal(0) > 22.5)",
    "[]((signal(2) == 3) -> signal(0) > 25)",
    "[]((signal(2) == 3) -> signal(0) > 27.5)",
    "[]((signal(2) == 3) -> signal(0) > 30)"
).map { stlString ->
    stlFactory.parse(
        stlString,
        inputMapper,
        outputMapperReader.outputMapper,
        outputMapperReader.largest
    )
}.toList()
val signalLength = 30
val properties = AdaptiveSTLList(stlList, signalLength)

## Configure the verifier

In [10]:
val verifier = NumericSULVerifier(sul, signalStep, properties, mapper)

// Timeout must be set before adding equivalence testing
verifier.setTimeout(5 * 60) // 5 minutes

### Configure the equivalence testing

In this demo, we use the equivalence testing based on an genetic algorithm. The following defines the constants.

In [11]:
// Constants for the GA-based equivalence testing
val maxTest = 50000
val populationSize = 200
val crossoverProb = 0.5
val mutationProb = 0.01

verifier.addGAEQOracleAll(
    signalLength,
    maxTest,
    ArgParser.GASelectionKind.Tournament,
    populationSize,
    crossoverProb,
    mutationProb
)

## Run the verifier

Then, we run the verifier. This takes some minutes.

In [12]:
val result = verifier.run()
result

false

In [15]:
import net.automatalib.word.Word;

var rawOutput = mutableListOf<List<List<Double>>>()
for (i in 0 until verifier.cexProperty.size) {
    val dim = mutableListOf<List<Double>>()
    for (j in 0 until verifier.cexConcreteInput[i].size()) {
        dim.add(verifier.cexConcreteInput[i].get(j))
    }
    val inputWord = Word.fromList(dim)
    val resultWord = sul.execute(inputWord).getOutputSignal()
    rawOutput.add(resultWord.asList())
}

## Print the result

In [16]:
// Print the result
for (i in 0 until verifier.cexProperty.size) {
    println("${verifier.cexProperty[i]} is falsified by the following counterexample)")
    println("cex concrete input: ${verifier.cexConcreteInput[i]}")
    println("cex abstract input: ${verifier.cexAbstractInput[i]}")
    println("cex concrete output: ${rawOutput[i]}")
    println("cex abstract output: ${verifier.cexOutput[i]}")
}

[] ( ( output(2) == 3.000000 ) -> ( output(0) > 22.500000 ) ) is falsified by the following counterexample)
cex concrete input: [0.0 100.0 325.0; 1.0 0.0 325.0; 2.0 100.0 0.0; 3.0 0.0 325.0; 4.0 0.0 325.0; 5.0 0.0 325.0; 6.0 0.0 325.0; 7.0 0.0 0.0]
cex abstract input: bb ab ba ab ab ab ab aa
cex concrete output: [[0.0, 1000.0, 1.0], [19.118767463071062, 2789.630920727734, 1.0], [17.850883193685398, 603.719805211533, 2.0], [30.00786996251345, 3572.538441674258, 1.0], [28.797025783906747, 600.0, 3.0], [26.553765135572213, 600.0, 3.0], [24.325155569416047, 600.0, 3.0], [22.110909928456, 600.0, 3.0]]
cex abstract output: aaa aaa aaa faa eab dab cab bab
[] ( ( output(2) == 3.000000 ) -> ( output(0) > 25.000000 ) ) is falsified by the following counterexample)
cex concrete input: [0.0 100.0 325.0; 1.0 0.0 325.0; 2.0 100.0 0.0; 3.0 0.0 325.0; 4.0 0.0 325.0; 5.0 0.0 325.0; 6.0 0.0 0.0]
cex abstract input: bb ab ba ab ab ab aa
cex concrete output: [[0.0, 1000.0, 1.0], [19.118767463071062, 2789.

## Plot the result

We can also plot the result of the falsification using `lets-plot`. We remark that we use the discrete-time semantics, and the results shown here are also rather coarsely sampled (by default, 1.0-time unit).

In [17]:
%use lets-plot

In [18]:
val bunch = GGBunch()
    for (i in 0 until verifier.cexProperty.size) {
        val size = verifier.cexConcreteInput[i].size()
        val datasetIn = mapOf(
            "time" to List(size) { it.toDouble() } + List(size) { it.toDouble() },
            "input" to verifier.cexConcreteInput[i].dimensionGet(0) + verifier.cexConcreteInput[i].dimensionGet(1),
            "group" to List(size) {"throttle"} + List(size) {"brake"}
        )
        bunch.addPlot(letsPlot(datasetIn) + 
                geomPath(showLegend = true) {x = "time"; y = "input"; color = "group"} +
                labs(title = "Input to falsify" + verifier.cexProperty[i]), 0, 800 * i, 1000, 200)
        val datasetOut = mapOf(
            "time" to (0 until size).flatMap { value -> List(3) { value.toDouble() } },
            "output" to rawOutput[i].flatten(),
            "group" to (1..size + 1).flatMap { listOf("velocity", "rotation", "gear") }.take(size * 3)
        )
        bunch.addPlot(letsPlot(datasetOut) + 
                geomPath(showLegend = true) {x = "time"; y = "output"; color = "group"} +
                labs(title = "Output to falsify" + verifier.cexProperty[i]), 0, 800 * i + 200, 1000, 200)
        val indicesToExclude = datasetOut["group"]!!.withIndex().filter { it.value == "rotation" }.map { it.index }
        val noRotation = datasetOut.mapValues { (key, value) ->
            if (value is List<*>) value.withIndex().filterNot { it.index in indicesToExclude }.map { it.value }
            else value
        }
        bunch.addPlot(letsPlot(noRotation) + 
                geomPath(showLegend = true) {x = "time"; y = "output"; color = "group"} +
                labs(title = "Output to falsify" + verifier.cexProperty[i] + " (without rotation for visualization)"), 0, 800 * i + 400, 1000, 200)
        val gearIndices = datasetOut["group"]!!.withIndex().filter { it.value == "gear" }.map { it.index }
        val onlyGear = datasetOut.mapValues { (key, value) ->
            if (value is List<*>) value.withIndex().filter { it.index in gearIndices }.map { it.value }
            else value
        }
        bunch.addPlot(letsPlot(onlyGear) + 
                geomPath(showLegend = true) {x = "time"; y = "output"; color = "group"} +
                labs(title = "Output to falsify" + verifier.cexProperty[i] + " (only gear for visualization)"), 0, 800 * i + 600, 1000, 200)
    }
bunch.show()

## IMPORTANT!! Close MATLAB engine

The following terminates the MATLAB engine. This must be executed at the end. Otherwise, the MATLAB process remains running. If you want to run the Simulink model, you can just re-initialize it.

In [19]:
sul.close()